In [50]:
import pandas as pd
from glob import glob

In [51]:
!pip3 install xlrd

In [52]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [53]:
filepaths_order = glob('sources/order_new/order*.xlsx')
filepaths_order

[]

In [54]:
filepath = filepaths_order[0]
filepath

IndexError: list index out of range

In [ ]:
df_order = pd.read_excel(filepath)
df_order

In [ ]:
df_order.to_dict().items()

In [ ]:
from collections import defaultdict

In [ ]:
order = defaultdict(int)

In [ ]:
for key, value in df_order.to_dict().items():
    order[key] += value[0]
#     print(key, value) 

In [ ]:
order

In [ ]:
filepath = filepaths_order[1]
df_order = pd.read_excel(filepath)
df_order

In [ ]:
for key, value in df_order.to_dict().items():
    order[key] += value[0]

In [ ]:
order

In [ ]:
import os
import shutil

In [ ]:
os.listdir('sources/order_new/')

In [ ]:
os.listdir('sources/order_old/')

In [ ]:
filepath = filepaths_order[0]
shutil.move(filepath, 'sources/order_old/')

In [ ]:
def update_order(order, filepath):
    # 注文表読み込み
    df_order = pd.read_excel(filepath)
    for key, value in df_order.to_dict().items():
        order[key] += value[0]
    # ファイルの移動
    shutil.move(filepath, 'sources/order_old/')
    return order

In [ ]:
order = defaultdict(int)

for filepath in filepaths_order:
    order = update_order(order, filepath)

In [ ]:
order

In [ ]:
order

In [56]:
filepath_stock = 'sources/stock.xlsx'
df_stock = pd.read_excel(filepath_stock)

df_stock.tail(3)

,日付,曜日,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
12,2021-02-02,Tue,66,48,38,70,31,18,98
13,2021-02-03,Wed,66,32,34,16,24,16,75
14,2021-02-03,Wed,66,16,30,-38,17,14,52


In [ ]:
pd.DataFrame([[10, 4, 6]])

In [ ]:
pd.DataFrame([[5, 1, 10]])

In [ ]:
pd.DataFrame([[10, 4, 6]]) - pd.DataFrame([[5, 1, 10]])

In [57]:
stock = df_stock.iloc[-1:, 2:]
stock

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
14,66,16,30,-38,17,14,52


In [58]:
order = pd.DataFrame(order, index=[0], columns=stock.columns)
order

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
order = order.fillna(0)
order

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,0,0,0,0,0,0,0


In [60]:
stock - order

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
stock = stock.reset_index(drop=True)

In [62]:
updated = stock - order
updated

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,66,16,30,-38,17,14,52


In [63]:
filepath_master = 'sources/master.xlsx'
df_master = pd.read_excel(filepath_master)
df_master

,Unnamed: 0,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,閾値,30,20,20,20,30,10,40
1,追加量,100,80,100,100,80,80,120


In [64]:
df_master.iloc[0 , 1: ]

日本酒       30
焼酎        20
赤ワイン      20
白ワイン      20
ウィスキー     30
ブランデー     10
缶チューハイ    40
Name: 0, dtype: object

In [65]:
threshold = df_master.iloc[:1 , 1: ]
threshold

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,30,20,20,20,30,10,40


In [66]:
updated

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,66,16,30,-38,17,14,52


In [69]:
shortage_columns = updated[updated < threshold].dropna(axis=1).columns
shortage_columns

Index(['焼酎', '白ワイン', 'ウィスキー'], dtype='object')

In [72]:
df_shortage = df_master.iloc[1:, 1:][shortage_columns]
df_shortage

,焼酎,白ワイン,ウィスキー
1,80,100,80


In [75]:
order_text = ''
for key, value in df_shortage.to_dict().items():
    order_text += f'{key}を{value[1]}本、'
order_text += '発注お願いします。'

In [76]:
order_text

'焼酎を80本、白ワインを100本、ウィスキーを80本、発注お願いします。'